In [16]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

# Load the first dataset
data_path_1 = r'C:\Users\admin\Downloads\two_pot_retirement_system_dataset.csv'
data_1 = pd.read_csv(data_path_1)

# Load the second dataset
data_path_2 = r'C:\Users\admin\Downloads\retirement_savings_data.csv'
data_2 = pd.read_csv(data_path_2)

# Display the first few rows of the first dataset
print("Dataset 1 Overview:")
print(data_1.head())

# Display the first few rows of the second dataset
print("\nDataset 2 Overview:")
print(data_2.head())

# Print actual column names for both datasets
print("\nDataset 1 Column Names:")
print(data_1.columns.tolist())
print("\nDataset 2 Column Names:")
print(data_2.columns.tolist())

# Data Cleaning for Dataset 1
# Handle missing values
imputer = SimpleImputer(strategy='mean')  # Mean strategy for numerical columns

# Impute missing values for numeric columns in dataset 1
for column in data_1.select_dtypes(include=[np.number]).columns:
    if data_1[column].isnull().any():
        data_1[column] = imputer.fit_transform(data_1[[column]])

# Fill missing values for non-numeric columns in dataset 1
for column in data_1.select_dtypes(exclude=[np.number]).columns:
    if data_1[column].isnull().any():
        data_1[column] = data_1[column].fillna(data_1[column].mode()[0])

# Remove duplicates in Dataset 1
data_1 = data_1.drop_duplicates()

# Data Cleaning for Dataset 2
# Handle missing values
for column in data_2.select_dtypes(include=[np.number]).columns:
    if data_2[column].isnull().any():
        data_2[column] = imputer.fit_transform(data_2[[column]])

# Fill missing values for non-numeric columns in dataset 2
for column in data_2.select_dtypes(exclude=[np.number]).columns:
    if data_2[column].isnull().any():
        data_2[column] = data_2[column].fillna(data_2[column].mode()[0])

# Remove duplicates in Dataset 2
data_2 = data_2.drop_duplicates()

# Initialize merged_data as an empty DataFrame
merged_data = pd.DataFrame()

# Check for common columns to merge on
common_columns = set(data_1.columns).intersection(set(data_2.columns))
print("\nCommon Columns for Merging:")
print(common_columns)

# Ensure that there are common columns before attempting to merge
if common_columns:
    # Replace 'common_column_name' with the actual name of the common column you want to use for merging
    common_column_name = 'your_common_column_name_here'  # Update this with the actual column name

    # Merge the two datasets (assuming they can be merged on a common column)
    try:
        merged_data = pd.merge(data_1, data_2, on=common_column_name, how='outer')  # Use 'inner' for inner join
        print("\nDatasets merged successfully.")
    except KeyError as e:
        print(f"Error merging datasets: {e}. Check if '{common_column_name}' exists in both datasets.")
else:
    print("No common columns found to merge the datasets.")

# Proceed with calculations only if merged_data is not empty
if not merged_data.empty:
    # Example: Feature Engineering - Creating a savings to income ratio
    savings_column = 'actual_savings_column'  # Update this with your savings column name
    income_column = 'actual_income_column'  # Update this with your income column name

    # Check if the columns exist before calculating the ratio
    if savings_column in merged_data.columns and income_column in merged_data.columns:
        merged_data['savings_to_income'] = merged_data[savings_column] / merged_data[income_column]
        print(f"\n'Savings to Income' ratio calculated successfully.")
    else:
        missing_cols = [col for col in [savings_column, income_column] if col not in merged_data.columns]
        print(f"Columns not found in the dataset: {missing_cols}")

    # Visualizations
    if 'savings_to_income' in merged_data.columns:
        plt.figure(figsize=(10, 6))
        plt.hist(merged_data['savings_to_income'].dropna(), bins=20, color='skyblue', edgecolor='black')
        plt.title('Distribution of Savings to Income Ratio')
        plt.xlabel('Savings to Income Ratio')
        plt.ylabel('Frequency')
        plt.grid(axis='y', alpha=0.75)
        plt.show()
    else:
        print("The 'savings_to_income' column was not created, skipping histogram.")

    # Convert non-numeric columns to numeric using one-hot encoding (if needed)
    merged_data = pd.get_dummies(merged_data, drop_first=True)

    # Now calculate the correlation matrix
    correlation_matrix = merged_data.corr()
    print("\nCorrelation Matrix:")
    print(correlation_matrix)

    # Preparing Data for Machine Learning
    # Dropping non-numeric columns for machine learning (or encode them)
    numeric_data = merged_data.select_dtypes(include=[np.number])
    X = numeric_data.drop('savings_to_income', axis=1, errors='ignore')  # Features
    y = numeric_data['savings_to_income'] if 'savings_to_income' in numeric_data.columns else None  # Target

    # Display the prepared features and target
    print("\nFeatures (X):")
    print(X.head())
    print("\nTarget (y):")
    if y is not None:
        print(y.head())
    else:
        print("Target variable 'savings_to_income' is not available for training.")
else:
    print("No data available for further analysis after merging.")


Dataset 1 Overview:
   User_ID  Age  Gender Employment_Status Income_Level         Region  \
0        1   53  Female          Employed  30k-40k ZAR   Western Cape   
1        2   39    Male          Employed  40k-50k ZAR  KwaZulu-Natal   
2        3   32    Male          Employed  10k-20k ZAR   Eastern Cape   
3        4   45  Female          Employed  10k-20k ZAR        Gauteng   
4        5   43    Male          Employed  40k-50k ZAR        Gauteng   

  Contribution_Date  Total_Contribution  Savings_Pot_Contribution  \
0        2023-05-10                2757                       919   
1        2023-02-14                1356                       452   
2        2023-01-30                3070                      1023   
3        2023-12-10                2785                       928   
4        2023-05-20                3569                      1189   

   Retirement_Pot_Contribution  ... Withdrawal_Amount  Withdrawal_Reason  \
0                         1838  ...               

In [17]:


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# Load the datasets
data_1 = pd.read_csv(r'C:\Users\admin\Downloads\two_pot_retirement_system_dataset.csv')
data_2 = pd.read_csv(r'C:\Users\admin\Downloads\retirement_savings_data.csv')

# Print columns to verify the datasets
print("Dataset 1 Columns:", data_1.columns.tolist())
print("Dataset 2 Columns:", data_2.columns.tolist())

# Identify a common column for merging - update with the actual common column name
common_column_name = 'User_ID'  # Replace with the actual common column name (e.g., 'User_ID')

# Check if the common column exists in both datasets before merging
if common_column_name in data_1.columns and common_column_name in data_2.columns:
    # Merge the datasets
    merged_data = pd.merge(data_1, data_2, on=common_column_name, how='outer')
    
    # Print the first few rows of the merged dataset
    print("Merged Dataset Preview:")
    print(merged_data.head())
else:
    print(f"The common column '{common_column_name}' does not exist in both datasets.")

# Data cleaning
merged_data.ffill(inplace=True)  # Use forward fill to handle missing values

# Assuming there's a target variable to encode, adjust this according to your dataset
if 'withdrawal' not in merged_data.columns:
    print("Warning: 'withdrawal' column not found in the merged dataset.")
    # If you have a different target variable, replace 'withdrawal' with the correct column name
    # raise KeyError("'withdrawal' column not found in the merged dataset. Please check the dataset.")

# Encoding the target variable (adjust according to your dataset)
# Example: merged_data['withdrawal'] = merged_data['withdrawal'].map({'yes': 1, 'no': 0})

# Splitting features and target variable (if a target variable exists)
# Example: X = merged_data.drop(columns=['withdrawal'], errors='ignore')  # Features
# Example: y = merged_data['withdrawal']  # Target variable

# Train-test split
# Example: X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
# Example: scaler = StandardScaler()
# Example: X_train_scaled = scaler.fit_transform(X_train)
# Example: X_test_scaled = scaler.transform(X_test)

# Logistic Regression Model (uncomment when you have a target variable)
# model = LogisticRegression()
# model.fit(X_train_scaled, y_train)

# Predictions (uncomment when you have a target variable)
# y_pred = model.predict(X_test_scaled)

# Evaluation (uncomment when you have a target variable)
# print("Confusion Matrix:")
# print(confusion_matrix(y_test, y_pred))
# print("\nClassification Report:")
# print(classification_report(y_test, y_pred))
# roc_auc = roc_auc_score(y_test, model.predict_proba(X_test_scaled)[:, 1])
# print(f"ROC-AUC Score: {roc_auc:.2f}")


Dataset 1 Columns: ['User_ID', 'Age', 'Gender', 'Employment_Status', 'Income_Level', 'Region', 'Contribution_Date', 'Total_Contribution', 'Savings_Pot_Contribution', 'Retirement_Pot_Contribution', 'Withdrawal_Date', 'Withdrawal_Amount', 'Withdrawal_Reason', 'Withdrawal_Frequency', 'Remaining_Balance', 'Interest_Rate', 'Annual_Return', 'Retirement_Balance', 'Projected_Retirement_Amount', 'Retirement_Age', 'Expected_Annual_Income_Post_Retirement']
Dataset 2 Columns: ['User_ID', 'Age', 'Gender', 'Income_Level', 'Region', 'Contribution_Date', 'Total_Contribution', 'Savings_Pot_Contribution', 'Retirement_Pot_Contribution', 'Interest_Rate', 'Annual_Return', 'Retirement_Balance', 'Projected_Retirement_Amount']
Merged Dataset Preview:
   User_ID  Age_x Gender_x Employment_Status Income_Level_x       Region_x  \
0        1     53   Female          Employed    30k-40k ZAR   Western Cape   
1        2     39     Male          Employed    40k-50k ZAR  KwaZulu-Natal   
2        3     32     Male   

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller

# Load the retirement savings dataset
savings_data = pd.read_csv(r'C:\Users\admin\Downloads\retirement_savings_data.csv')
print("First few rows of the retirement savings dataset:")
print(savings_data.head())
print("Columns in retirement savings dataset:", savings_data.columns.tolist())
input("Press Enter to continue after checking the column names...")

# Specify the date column name
date_column_name_savings = 'Date'  # Replace with the actual date column name
savings_data[date_column_name_savings] = pd.to_datetime(savings_data[date_column_name_savings])
savings_data.set_index(date_column_name_savings, inplace=True)

# Plot historical data for retirement savings
plt.figure(figsize=(10, 5))
plt.plot(savings_data['Retirement'], label='Retirement Savings')  # Replace 'Retirement' with the actual column name
plt.title('Historical Retirement Savings')
plt.xlabel('Date')
plt.ylabel('Savings')
plt.legend()
plt.show()

# Check for stationarity and differencing if necessary
result_savings = adfuller(savings_data['Retirement'])
print('ADF Statistic for retirement savings:', result_savings[0])
print('p-value for retirement savings:', result_savings[1])

# Differencing
savings_data['diff'] = savings_data['Retirement'].diff()
savings_data['diff'].dropna().plot(figsize=(10, 5), title='Differenced Retirement Savings Data')
plt.show()

# Fit the ARIMA model for retirement savings
model_savings = ARIMA(savings_data['Retirement'], order=(1, 1, 1))  # Adjust (p, d, q) as needed
model_fit_savings = model_savings.fit()
print("ARIMA Model Summary for Retirement Savings:")
print(model_fit_savings.summary())

# Forecast retirement savings
forecast_savings = model_fit_savings.forecast(steps=12)
plt.figure(figsize=(10, 5))
plt.plot(savings_data['Retirement'], label='Historical Savings')
plt.plot(forecast_savings, label='Forecast', color='red')
plt.title('Retirement Savings Forecast')
plt.xlabel('Date')
plt.ylabel('Savings')
plt.legend()
plt.show()


# Load the two pot retirement system dataset
two_pot_data = pd.read_csv(r'C:\Users\admin\Downloads\two_pot_retirement_system_dataset.csv')
print("First few rows of the two pot retirement system dataset:")
print(two_pot_data.head())
print("Columns in two pot retirement dataset:", two_pot_data.columns.tolist())
input("Press Enter to continue after checking the column names...")

# Specify the date column name
date_column_name_two_pot = 'Date'  # Replace with the actual date column name
two_pot_data[date_column_name_two_pot] = pd.to_datetime(two_pot_data[date_column_name_two_pot])
two_pot_data.set_index(date_column_name_two_pot, inplace=True)

# Plot historical data for two pot savings
plt.figure(figsize=(10, 5))
plt.plot(two_pot_data['Savings'], label='Two Pot Savings')  # Replace 'Savings' with the actual column name
plt.title('Historical Two Pot Savings')
plt.xlabel('Date')
plt.ylabel('Savings')
plt.legend()
plt.show()

# Check for stationarity and differencing if necessary
result_two_pot = adfuller(two_pot_data['Savings'])
print('ADF Statistic for two pot savings:', result_two_pot[0])
print('p-value for two pot savings:', result_two_pot[1])

# Differencing
two_pot_data['diff'] = two_pot_data['Savings'].diff()
two_pot_data['diff'].dropna().plot(figsize=(10, 5), title='Differenced Two Pot Savings Data')
plt.show()

# Fit the ARIMA model for two pot savings
model_two_pot = ARIMA(two_pot_data['Savings'], order=(1, 1, 1))  # Adjust (p, d, q) as needed
model_fit_two_pot = model_two_pot.fit()
print("ARIMA Model Summary for Two Pot Savings:")
print(model_fit_two_pot.summary())

# Forecast two pot savings
forecast_two_pot = model_fit_two_pot.forecast(steps=12)
plt.figure(figsize=(10, 5))
plt.plot(two_pot_data['Savings'], label='Historical Savings')
plt.plot(forecast_two_pot, label='Forecast', color='red')
plt.title('Two Pot Savings Forecast')
plt.xlabel('Date')
plt.ylabel('Savings')
plt.legend()
plt.show()


First few rows of the dataset:
   User_ID  Age  Gender Income_Level         Region Contribution_Date  \
0        1   53  Female  10k-20k ZAR  KwaZulu-Natal        2023-02-28   
1        2   39    Male  10k-20k ZAR  KwaZulu-Natal        2023-08-08   
2        3   32    Male  10k-20k ZAR        Gauteng        2023-02-07   
3        4   45  Female  40k-50k ZAR        Gauteng        2023-08-10   
4        5   43    Male  10k-20k ZAR  KwaZulu-Natal        2024-01-01   

   Total_Contribution  Savings_Pot_Contribution  Retirement_Pot_Contribution  \
0                3146                      1048                         2098   
1                4963                      1654                         3309   
2                2768                       922                         1846   
3                2060                       686                         1374   
4                3327                      1109                         2218   

   Interest_Rate  Annual_Return  Retirement_Balan